Import the csv fromt he donwloads folder of the Supercias website

In [15]:
packages = ['seaborn', 'matplotlib', 'numpy', 'altair', 'pandas', 'numpy', 'os']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import ScalarFormatter
import altair as alt
import os 

# set the default renderer to vega
alt.data_transformers.enable("vegafusion")
alt.renderers.enable('default')
# Set display format for floating-point numbers
pd.options.display.float_format = '{:,.2f}'.format

#check if the following files are in the Downloads folder
domain = 'https://appscvsmovil.supercias.gob.ec/ranking/recursos/'
folder = '/home/telix/Downloads/'
ranking_path = folder + 'bi_ranking.csv' if 'bi_ranking.csv' in os.listdir(folder) else domain + 'bi_ranking.csv'
ids_path = folder + 'bi_compania.csv' if 'bi_compania.csv' in os.listdir(folder) else domain + 'bi_compania.csv'
segmentos_path = folder + 'bi_segmento.csv' if 'bi_segmento.csv' in os.listdir(folder) else domain + 'bi_segmento.csv'
ciiu_path = folder + 'bi_ciiu.csv' if 'bi_ciiu.csv' in os.listdir(folder) else domain + 'bi_ciiu.csv'

# read from a csv file into a pd dataframe
df_ranking = pd.read_csv(ranking_path, low_memory=False)
# the companias string 
df_ids = pd.read_csv(ids_path, low_memory=False)
# segementos
df_segmentos = pd.read_csv(segmentos_path, low_memory=False)
# Código de Clasificacón Industrial Internacional Unifrome
df_ciiu = pd.read_csv(ciiu_path, low_memory=False)

# let merge the df into one dataframe and only select the 2023 data
df_ranking = df_ranking[df_ranking['anio'] == 2023]

# rename the cuii column 
df_ciiu = df_ciiu.rename(columns={'descripcion': 'ciiu_desc', 'ciiu': 'ciiu_code'}) 
df_ciiu['ciiu_code'] = df_ciiu['ciiu_code'].str.strip()

# match all of the expedientes in the df_ids with the expedientes in the df_ranking 
df = pd.merge(df_ranking, df_ids, on='expediente', how='left')

# let's merge the ciiu and the segments with the 
df = pd.merge(df, df_ciiu, left_on='ciiu_n1', right_on='ciiu_code', how='left')
df.rename(columns={'ciiu_code': 'ciiu_n1_code', 'ciiu_desc': 'ciiu_n1_desc'}, inplace=True)
df = pd.merge(df, df_ciiu, left_on='ciiu_n6', right_on='ciiu_code', how='left')
df.rename(columns={'ciiu_code': 'ciiu_n6_code', 'ciiu_desc': 'ciiu_n6_desc'}, inplace=True)

# make companies categories between 0 and 2.5 billion in assets
# small companies: 0 - 500 thousand
# medium companies: 500 thousand - 50 million
# large companies: 50 million - up 
df_small = df[df['activos'] < 500000]
df_medium = df[(df['activos'] >= 500000) & (df['activos'] < 50000000)]
df_large = df[(df['activos'] >= 50000000)]


We also want to read from the the mondo BD and get all fo teh documets from each company

In [16]:
from pymongo import MongoClient
from gridfs import GridFS

# Connect to MongoDB
db = MongoClient('mongodb://10.0.10.5:27017')['supercias']
companies = db['companies']

# find one
comp = companies.find_one()
#print(comp['Documentos online'])
pdfs = []
# if company has documents
for value in comp['Documentos online'].values():
    if isinstance(value, list) and len(value) > 0:
        [ pdfs.append(v) for v in value ]

# Access the GridFS collection
fs = GridFS(db, collection='companies')

For every large company we will try to get the online document from each one

In [27]:
index = 5
df_large[['ruc', 'nombre']]

# the the first row
ruc = df_large[['ruc', 'nombre']].iloc[index]['ruc']
nombre = df_large[['ruc', 'nombre']].iloc[index]['nombre']
year = '2023'
pdf_title = ''
print(nombre)


# get all of the files with the ruc in the filename
pdf_queries = fs.find({'filename': {'$regex': f'.*{ruc}.*{pdf_title}.*{year}.*'}})

# print names of files

count = 0
for pdf in pdf_queries:
    print(pdf.filename)
    count += 1
print('Total:', count)
#print('found:', list(pdf_queries))


# Retrieve all files from GridFS collection that have in the filen ethe substring 'Balance  Estado de Situación'
#pdf_queries = fs.find({'filename': {'$regex': '.*Balance  Estado de Situación Financiera_2022.*'}})


CORPORACION EL ROSADO S.A.
Total: 0
